# Amazon SageMaker Model Monitoring 

ML Monitoring is a critical MLOps capability to reduce risk and manage safe and reliable production machine learning systems at scale. SageMaker contains several integrated services such as [SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) and [SageMaker Clarify](https://aws.amazon.com/sagemaker/clarify/) to monitor models for data and model quality, bias, and feature attribution drift.

Amazon SageMaker Model Monitor continuously monitors the quality of Amazon SageMaker machine learning models in production. With Model Monitor, you can set alerts that notify you when there are deviations in the model quality. Early and proactive detection of these deviations enables you to take corrective actions, such as retraining models, auditing upstream systems, or fixing quality issues without having to monitor models manually or build additional tooling. Model Monitor integrates with SageMaker Clarify to provide pre-built and extendable monitors to get start with monitoring your ML models faster.

In this lab, you will learn how to:
  * Capture inference requests, results, and metadata from our pipeline deployed model.
  * Schedule a data monitor to monitor for data drift on a regular schedule.

While each monitor requires task-specific configurations, the standardized monitoring setup workflow you will follow is:

1. Initialize a monitoring object
2. Configure and run a baseline job to contrastively compare results
3. Schedule continuous monitoring

The goal of this lab is that you walk through the code and understand how get started with monitoring your machine learning models with SageMaker Model Monitor. 

## Setup

In [155]:
!pip install "sagemaker>=2.117.0"

In [3]:
from datetime import datetime, timedelta
import pandas as pd
import time
import csv
import json
import boto3
import sagemaker

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
    DefaultModelMonitor,
    ModelQualityMonitor,
)

from sagemaker.model_monitor.dataset_format import DatasetFormat

from sagemaker.s3 import S3Downloader, S3Uploader

In [4]:
print(f"AWS region: {region}")
# A different bucket can be used, but make sure the role for this notebook has
# the s3:PutObject permissions. This is the bucket into which the data is captured.
print(f"S3 Bucket: {default_bucket}")

# Endpoint metadata.
# Note: you will use the staging endpoint from the previously lab just as you would in a real scenario to verify your monitoring
# setup before deploying your setup on production endpoints.
endpoint_name = "workshop-project-staging"
endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.large"
print(f"Endpoint: {endpoint_name}")

prefix = "sagemaker/xgboost-dm-model-monitoring"
s3_key = f"s3://{default_bucket}/{prefix}"
print(f"S3 key: {s3_key}")

s3_capture_upload_path = f"{s3_key}/data_capture"
s3_baseline_results_path = f"{s3_key}/baselines"
s3_report_path = f"{s3_key}/reports"

print(f"Capture path: {s3_capture_upload_path}")
print(f"Baselines path: {s3_baseline_results_path}")
print(f"Report path: {s3_report_path}")

sm_client = boto3.client('sagemaker')

endpoint_config = sm_client.describe_endpoint(EndpointName = endpoint_name)['EndpointConfigName']
model_name = sm_client.describe_endpoint_config(EndpointConfigName = endpoint_config)['ProductionVariants'][0]['ModelName']

print("Model Name : ", model_name)

AWS region: ap-south-1
S3 Bucket: sagemaker-ap-south-1-682101512330
Endpoint: workshop-project-staging
S3 key: s3://sagemaker-ap-south-1-682101512330/sagemaker/xgboost-dm-model-monitoring
Capture path: s3://sagemaker-ap-south-1-682101512330/sagemaker/xgboost-dm-model-monitoring/data_capture
Baselines path: s3://sagemaker-ap-south-1-682101512330/sagemaker/xgboost-dm-model-monitoring/baselines
Report path: s3://sagemaker-ap-south-1-682101512330/sagemaker/xgboost-dm-model-monitoring/reports
Model Name :  Model-w6SQK6sjI4LX


## Configure data capture and generate synthetic traffic

Data quality monitoring automatically monitors machine learning (ML) models in production and notifies you when data quality issues arise. ML models in production have to make predictions on real-life data that is not carefully curated like most training datasets. If the statistical nature of the data that your model receives while in production drifts away from the nature of the baseline data it was trained on, the model begins to lose accuracy in its predictions. Amazon SageMaker Model Monitor uses rules to detect data drift and alerts you when it happens.

### Initialize SageMaker Predictor for real-time requests to previously deployed model endpoint

In [5]:
# Create a Predictor Python object for real-time endpoint requests. https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html
predictor = Predictor(endpoint_name=endpoint_name, serializer=CSVSerializer())

In [6]:
# SageMaker automatically created a DataCaptureConfig when your model was deployed to an endpoint 
# in a prior lab that already had data capture enabled. Below is illustrating how create a custom 
# DataCaptureConfig with data capture enabled and update an existing endpoint.
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
)

In [7]:
# Now update endpoint with data capture enabled and provide an s3_capture_upload_path.
predictor.update_data_capture_config(data_capture_config)

-----!

Note: updating your endpoint data config can take 3-5 min. A progress bar will be displayed in the cell above and indicates completion with `---------------!` and the cell execution number. You will see your endpoint status as `Updating` under SageMaker resources > Endpoints while this is in progress and `InService` when your updated endpoint is ready for requests.

### Invoke the deployed model endpoint to generate predictions

Now send data to this endpoint to get inferences in real time. 

With data capture enabled in the previous step, the request and response payload, along with some additional metadata, is saved to the S3 location specified in `DataCaptureConfig`.

In [69]:
# Read in training set for schema and to compute feature attribution baselines.
train_df = pd.read_csv("train-headers.csv")

In [139]:
# Use test set to create a file without headers and labels to mirror data format at inference time.
test_df = pd.read_csv("test.csv", header = None)
test_df.drop(test_df.columns[0], axis=1, inplace=True)
test_df.sample(180).to_csv('test-samples-no-header.csv', header = False, index = None)

Now send a test batch of 180 requests to the model endpoint. These inputs will be captured along with endpoint output predictions and sent to your `s3_capture_upload_path`.

In [140]:
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

test_sample_df = pd.read_csv('test-samples-no-header.csv', header = None, index_col = False)

response = predictor.predict(data=test_sample_df.to_numpy())

print("Done!")

Sending test traffic to the endpoint workshop-project-staging. 
Please wait...
Done!


### View captured data

List the data capture files stored in Amazon S3. 

There should be different files from different time periods organized in S3 based on the hour in which the invocation occurred in the format: 

`s3://{destination-bucket-prefix}/{endpoint-name}/{AllTraffic or model-variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [141]:
print("Waiting 60 seconds for captures to show up", end="")

for _ in range(60):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}"))
    if capture_files:
        break
    print(".", end="", flush=True)
    time.sleep(1)

print("\nFound Capture Files:")
print("\n ".join(capture_files[-10:]))

Waiting 60 seconds for captures to show up........................................................
Found Capture Files:
s3://sagemaker-ap-south-1-682101512330/sagemaker/xgboost-dm-model-monitoring/data_capture/workshop-project-staging/AllTraffic/2022/12/07/23/22-01-065-e3264b34-2c1b-4744-9457-fa894c1e4dbd.jsonl


Next, view the content of a single capture file, looking at the first few lines in the captured file.

In [142]:
capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")[-10:-1]
print(capture_file[-1])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"48,661,3,999,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0\n38,63,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0\n38,80,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0\n50,252,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0\n51,527,2,999,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0\n29,143,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0\n27,490,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0\n46,417,6,999,0,1,0,0,0,0,0,0,0,

View a single line is present below in a formatted JSON file.

In [143]:
print(json.dumps(json.loads(capture_file[-1]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "48,661,3,999,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0\n38,63,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0\n38,80,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0\n50,252,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0\n51,527,2,999,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0\n29,143,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0\n27,490,2,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,

### Generate synthetic traffic

In order to review SageMaker's continuous monitoring capabilities, you will start a thread to generate synthetic traffic to send to the deployed model endpoint. 

The `WorkerThread` class will run continuously on the notebook kernel to generate predictions that are captured and sent to S3 until the kernel is restarted or the thread is explicitly terminated. 

See the cell in the `Cleanup` section to terminate the threads.

This step is necessary because if there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

This cell extends a Python Thread class to be able to able to terminate the thread later on without terminating the notebook kernel.

In [144]:
import threading

class WorkerThread(threading.Thread):
    def __init__(self, do_run, *args, **kwargs):
        super(WorkerThread, self).__init__(*args, **kwargs)
        self.__do_run = do_run
        self.__terminate_event = threading.Event()

    def terminate(self):
        self.__terminate_event.set()

    def run(self):
        while not self.__terminate_event.is_set():
            self.__do_run(self.__terminate_event)

Now you define a function that your thread will invoke continuously to send test samples to the model endpoint.

In [145]:
def invoke_endpoint(terminate_event):
    with open("test-samples-no-header.csv", "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )
            i += 1
            response["Body"].read()
            time.sleep(1)
            if terminate_event.is_set():
                break


# Keep invoking the endpoint with test data
invoke_endpoint_thread = WorkerThread(do_run=invoke_endpoint)
invoke_endpoint_thread.start()

## Monitor data quality

Data quality monitoring automatically monitors machine learning (ML) models in production and notifies you when data quality issues arise. ML models in production have to make predictions on real-life data that is not carefully curated like most training datasets. If the statistical nature of the data that your model receives while in production drifts away from the nature of the baseline data it was trained on, the model begins to lose accuracy in its predictions. Amazon SageMaker Model Monitor uses rules to detect data drift and alerts you when it happens.

### Configure `DefaultModelMonitor` for detecting data drift.

In [146]:
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)



SageMaker Model Monitor provides a built-in container that provides the ability to suggest the constraints automatically for CSV and flat JSON input. This sagemaker-model-monitor-analyzer container also provides you with a range of model monitoring capabilities, including constraint validation against a baseline, and emitting Amazon CloudWatch metrics. This container is based on Spark and is built with [Deequ "unit tests for data"](https://github.com/awslabs/deequ). All column names in your baseline dataset must be compliant with Spark. For column names, use only lowercase characters, and `_` as the only special character.

The training dataset that you used to trained the model is usually a good baseline dataset. The training dataset data schema and the inference dataset schema should exactly match (the number and order of the features). Note that the prediction/output columns are assumed to be the first columns in the training dataset. From the training dataset, you can ask SageMaker to suggest a set of baseline constraints and generate descriptive statistics to explore the data.

### Run data quality baseline job

Start a data quality baseline processing job with `DefaultModelMonitor.suggest_baseline(..)` using the Amazon SageMaker Python SDK. This uses an Amazon SageMaker Model Monitor prebuilt container that generates baseline statistics and suggests baseline constraints for the dataset and writes them to the `output_s3_uri` location that you specify.

The baseline calculations of statistics and constraints are needed as a standard against which data drift and other data quality issues can be detected.

Note: the baseline job will take about 5 min. If the log suppression does not work below, you can clear the cell output with `Edit > Clear Output`.

In [147]:
data_quality_baseline_job_name = f"DataQualityBaselineJob-{datetime.utcnow():%Y-%m-%d-%H%M}"

data_quality_baseline_job = data_quality_monitor.suggest_baseline(
    job_name=data_quality_baseline_job_name,
    baseline_dataset="train-headers.csv",
    dataset_format=DatasetFormat.csv(header=True),
)

data_quality_baseline_job.wait(logs=False)


Job Name:  DataQualityBaselineJob-2022-12-07-2323
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-682101512330/model-monitor/baselining/DataQualityBaselineJob-2022-12-07-2323/input/baseline_dataset_input', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-south-1-682101512330/model-monitor/baselining/DataQualityBaselineJob-2022-12-07-2323/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................2022-12-07 23:27:18,690 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2022-12-07 23:27:19.212405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libr

Amazon SageMaker Model Monitor prebuilt container computes per column/feature statistics. The statistics are calculated for the baseline dataset and also for the current dataset that is being analyzed. The cell below returns a table of computed baseline feature statistics.

In [148]:
latest_data_quality_baseline_job = data_quality_monitor.latest_baselining_job
schema_df = pd.json_normalize(latest_data_quality_baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,y_yes,Integral,28831,0,0.110749,3193.0,0.313821,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0,..."
1,age,Integral,28831,0,40.020672,1153836.0,10.369838,17.0,95.0,"[{'lower_bound': 17.0, 'upper_bound': 24.8, 'c...",0.64,2048.0,"[[27.0, 46.0, 50.0, 54.0, 46.0, 40.0, 47.0, 51..."
2,campaign,Integral,28831,0,2.574312,74220.0,2.770696,1.0,43.0,"[{'lower_bound': 1.0, 'upper_bound': 5.2, 'cou...",0.64,2048.0,"[[5.0, 1.0, 4.0, 3.0, 2.0, 2.0, 3.0, 2.0, 4.0,..."
3,pdays,Integral,28831,0,963.112102,27767485.0,185.333510,0.0,999.0,"[{'lower_bound': 0.0, 'upper_bound': 99.9, 'co...",0.64,2048.0,"[[999.0, 999.0, 999.0, 999.0, 999.0, 999.0, 99..."
4,previous,Integral,28831,0,0.171101,4933.0,0.493338,0.0,7.0,"[{'lower_bound': 0.0, 'upper_bound': 0.7, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0,..."
5,no_previous_contact,Integral,28831,0,0.963858,27789.0,0.186643,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
6,not_working,Integral,28831,0,0.086331,2489.0,0.280852,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,job_admin,Fractional,28831,0,0.252367,7276.0,0.434371,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,..."
8,job_blue-collar,Fractional,28831,0,0.225001,6487.0,0.417583,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
9,job_technician,Fractional,28831,0,0.163019,4700.0,0.369383,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


SageMaker Model Monitor will also suggest constraints per feature using the baseline statistics. View them using the cell below:

In [149]:
constraints_df = pd.json_normalize(latest_data_quality_baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,y_yes,Integral,1.0,True
1,age,Integral,1.0,True
2,campaign,Integral,1.0,True
3,pdays,Integral,1.0,True
4,previous,Integral,1.0,True
5,no_previous_contact,Integral,1.0,True
6,not_working,Integral,1.0,True
7,job_admin,Fractional,1.0,True
8,job_blue-collar,Fractional,1.0,True
9,job_technician,Fractional,1.0,True


### Schedule continuous data quality monitoring

You can create a data monitoring schedule for the endpoint created earlier. 

Use the baseline resources (constraints and statistics) to compare against the real-time traffic hourly.

In [150]:
## Create a data quality monitoring schedule name.
data_quality_monitor_schedule_name = (
    f"xgboost-dm-data-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"
)

In [151]:
# Create an EndpointInput
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    destination="/opt/ml/processing/input_data",
)

In [152]:
# Specify where to write the data quality monitoring results report to.
data_quality_baseline_job_result_uri = f"{s3_baseline_results_path}/data_quality"

response = data_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_quality_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=data_quality_baseline_job_result_uri,
    constraints=latest_data_quality_baseline_job.suggested_constraints(),
    # Create the monitoring schedule to execute every hour.    
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

In [153]:
# You will see the monitoring schedule in the 'Scheduled' status
data_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:ap-south-1:682101512330:monitoring-schedule/xgboost-dm-data-monitoring-schedule-2022-12-07-2329',
 'MonitoringScheduleName': 'xgboost-dm-data-monitoring-schedule-2022-12-07-2329',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2022, 12, 7, 23, 29, 30, 338000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 12, 7, 23, 29, 30, 402000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2022-12-07-23-29-30-066',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'workshop-project-staging',
 'ResponseMetadata': {'RequestId': '008faac4-1e70-4e63-850d-4d61b83e0816',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '008faac4-1e70-4e63-850d-4d61b83e0816',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '602',
   'da

In [154]:
# Check default model monitor created.
predictor.list_monitors()

In [ ]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to launch upto 20 min after the hour.
executions = data_quality_monitor.list_executions()
executions[:5]

## Cleanup

Well done! If you are finished with the notebook, run the following cells to terminate lab resources and prevent continued charges.

First, stop the worker threads.

In [ ]:
invoke_endpoint_thread.terminate()

Finally, stop and then delete all monitors scheduled to the endpoint.

If the following cell throws an error similar to `ClientError: An error occurred (ValidationException) when calling the DeleteMonitoringSchedule operation: can't delete schedule as it has in-progress executions`, wait a few minutes and run this cell again. You can't delete a monitor if a monitoring job is executing, once it is done, you can delete the monitoring schedule.

In [ ]:
model_monitors = predictor.list_monitors()

for monitor in model_monitors:
    monitor.stop_monitoring_schedule()
    monitor.delete_monitoring_schedule()